In [1]:
# Broker_vendor.py
import pandas as pd
import xml.etree.ElementTree as ET
import psycopg2
from datetime import datetime
import re
import cx_Oracle
# import numpy as np

# XML file path
xml_file_path = 'D:\\W\\RELEX\\RELEX\\Broker and Vendor\\Inbound Files\\VENDOR.xml'
# xml_file_path = 'D:\\W\\RELEX\\RELEX\\Broker and Vendor\\Inbound Files\\BROKER.xml'


# Open the XML file
with open(xml_file_path, 'r', encoding='utf-8') as file:
    xml_data = file.read()

# Database connection parameters
db_params = {
    "host": "gc-ue4-psql-cspo-dev01.nonprod.gcp.cswg.com",
    "database": "CSPODB",
    "port": 5432,
    "user": "cspoms",
    "password": "cspoms",
    "sslmode": "verify-ca",
    "sslcert": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\client-cert.pem",
    "sslkey": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\client-key.pem",
    "sslrootcert": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\server-ca.pem"
}

# Replace these values with your Oracle database connection details
username = 'apps'
password = 'apps'
host = 'csebsd2db.cswg.com' #EBSD2
port = '1521'
service_name = 'csebsd2_int' #EBSD2

# Create a connection
dsn = cx_Oracle.makedsn(host, port, service_name=service_name)
connection = cx_Oracle.connect(username, password, dsn)

# Table name
table_name = "cspoms.XXPO_ORCL2CSPOMS_VEND_STG".upper()
with psycopg2.connect(**db_params) as post_conn:
    with post_conn.cursor() as post_cur:
        # Get the nextval for request_id
        nextval_request_id = "SELECT nextval('cspoms.xxpo_cs_relex_run_id_seq')"
        post_cur.execute(nextval_request_id)
        request_id = post_cur.fetchone()[0]
        post_conn.commit()
        print(f"Request ID: {request_id}")

        # Get the current timestamp
        current_timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        print(f"Current Timestamp: {current_timestamp}")
# Split the XML data into lines
lines = xml_data.splitlines()

# Regular expression pattern to match the elements
pattern = re.compile(r"<([^>]+)>([^<]+)</\1>")
tag_to_column = {
    'BICEPSVENDOR': 'BIC_VENDOR_NUMBER',
    'BIC_VENDOR_FACILITY':'BIC_VENDOR_FACILITY',
    'BUYERNUMBER': 'BIC_BUYER_NUMBER',
    'CRPLEVEL': 'BIC_CRP_LEVEL',
    'STATUS': 'BIC_STATUS',
    'PORECOMMENDEDIND': 'BIC_PO_RECOMMENDED_IND',
    'AFETYPE1': 'BIC_AFE_TYPE1',
    'AFEFAXNUMBER1': 'BIC_AFE_FAX_NUMBER',
    'AFECONTACT1': 'BIC_AFE_CONTACT',
    'MINIMUMQUANTITY': 'BIC_MINIMUM_QUANTITY',
    'MINIMUMTYPE': 'BIC_MINIMUM_TYPE',
    'BKTQUANTITY1': 'BIC_BRACKET_QUANTITY',
    'CURRENTBKTNUMBER': 'BIC_CURRENT_BKT_NUMBER',
    'CURRENTBKTQUANTITY': 'BIC_CURRENT_BKT_QUANTITY',
    'CURRENTBKTTYPE': 'BIC_CURRENT_BKT_TYPE',
    'MAXIMUMQUANTITY': 'BIC_MAXIMUM_QUANTITY',
    'MAXIMUMTYPE': 'BIC_MAXIMUM_TYPE',
    'ORDERINTERVALWEEKS': 'BIC_ORDER_INTERVAL_WEEKS',
    'LEADTIMESTATEDWEEKS': 'BIC_LEAD_TIME_STATED_WEEKS',
    'FIXEDREVIEWDAY1': 'BIC_FIXED_REVIEW_DAY1',
    'FIXEDREVIEWDAY2': 'BIC_FIXED_REVIEW_DAY2',
    'FIXEDREVIEWDAY3': 'BIC_FIXED_REVIEW_DAY3',
    'FIXEDREVIEWDAY4': 'BIC_FIXED_REVIEW_DAY4',
    'FIXEDREVIEWDAY5': 'BIC_FIXED_REVIEW_DAY5',
    'FIXEDREVIEWDAY6': 'BIC_FIXED_REVIEW_DAY6',
    'FIXEDREVIEWDAY7': 'BIC_FIXED_REVIEW_DAY7',
    'TARGETSERVICELEVEL': 'BIC_TARGET_SERVICE_LEVEL',
    'NAME': 'BIC_NAME',
    'ADDRESS1': 'BIC_ADDRESS_1',
    'CITY': 'BIC_CITY',
    'STATE': 'BIC_STATE',
    'ZIP': 'BIC_ZIP',
    'PHONE': 'BIC_PHONE',
    'CONTACT': 'BIC_CONTACT',
    'TRANSFERMATCHTYPE': 'BIC_TRANSFER_MATCH_TYPE',
    'PAYABLEVENDORNBR': 'BIC_PAYABLE_VENDOR_NBR',
    'LEADTIMEWEEKSTYPE': 'BIC_LEAD_TIME_WEEKS_TYPE',
    'DUNSREMITNO': 'BIC_DUNS_REMIT_NO',
    'DUNSREMITSUFFIX': 'BIC_DUNS_REMIT_SUFFIX',
    'FACILITYSHIPTO': 'BIC_FACILITY_SHIP_TO',
    'PRESCHED': 'BIC_PRE_SCHED_FLAG',
    'BROKERNUMBER': 'BIC_BROKER_NUMBER',
    'FREIGHTALLOW': 'BIC_FREIGHT_ALLOW',
    'FREIGHTALLOWTYPE': 'BIC_FREIGHT_ALLOW_TYPE',
    'TERMSPERCENT': 'BIC_TERMS_PERCENT',
    'TERMSDAYS': 'BIC_TERMS_DAYS',
    'TERMSNETDAYS': 'BIC_TERMS_NET_DAYS',
    'TERMSBASE': 'BIC_TERMS_BASE',
    'BILLFORSHORTEDDEALSFLAG': 'BIC_BILL_FOR_SHORT_DEALS_FLAG',
    'TRANSITDAYS': 'BIC_TRANSIT_DAYS',
    'DUNSNO': 'BIC_DUNS_NO',
    '2NDMINQUANTITY': 'BIC_2ND_MIN_QUANTITY',
    '2NDMINTYPE': 'BIC_2ND_MIN_TYPE',
    '2NDMAXQUANTITY': 'BIC_2ND_MAX_QUANTITY',
    '2NDMAXTYPE': 'BIC_2ND_MAX_TYPE',
    'FLAGPREPAID': 'BIC_FLAG_PREPAID',
    'FLAGPREPAYANDADD': 'BIC_FLAG_PREPAY_AND_ADD',
    'FLAGFREIGHTBILL': 'BIC_FLAG_FREIGHT_BILL',
    'FLAGBACKHAUL': 'BIC_FLAG_BACKHAUL',
    'FLAGTRUCK': 'BIC_FLAG_TRUCK',
    'FLAGRAIL': 'BIC_FLAG_RAIL',
    'BACKHAUL': 'BIC_BACKHAUL',
    'BACKHAULTYPE': 'BIC_BACKHAUL_TYPE',
    'FREIGHTBILL': 'BIC_FREIGHTBILL',
    'ORDERSEQUENCE': 'BIC_ORDER_SEQUENCE',
    'SSAALLOWPERCENT': 'BIC_SSA_ALLOW_PERCENT',
    'SSAALLOWBASIS': 'BIC_SSA_ALLOW_BASIS',
    'SSAALLOWACCOUNT': 'BIC_SSA_ALLOW_ACCOUNT',
    'TRANSACTIONID': 'BIC_TRANSACTION_ID',
    'COUNTRY': 'BIC_COUNTRY',
    'ORDERFILTER': 'BIC_FILTER_VENDOR_FLAG',
    'LBLIMIT': 'BIC_LOAD_BLDG_LIMIT_FLAG',
    'WAREHOUSECODE': 'BIC_WAREHOUSE_CODE',
    'FLAGEXTBCKHL': 'BIC_EXTERNAL_BACKHAUL'
}
print('tag_to_column : ',len(tag_to_column))
# Loop through each line in the XML data
for line in lines:
    # Skip empty lines or lines containing only whitespace
    if not line.strip():
        continue

    # Define a dictionary to store values
    element_values = {}

    # Find all matches in the text data
    matches = re.findall(pattern, line)

    # Store the matches in the dictionary
    for tag, value in matches:
        element_values[tag] = value.strip()
    # Assuming 'BICEPSVENDOR' is present in element_values
    if 'BICEPSVENDOR' in element_values:
        bicepsvendor_value = element_values['BICEPSVENDOR']

        # Check if the value has at least 3 characters
        if len(bicepsvendor_value) >= 3:
            # Extract the first 3 digits
            facility_value = bicepsvendor_value[:3]

            # Assign the first 3 digits to a new key
            element_values['BIC_VENDOR_FACILITY'] = facility_value

            # Update 'BICEPSVENDOR' with the remaining value
            element_values['BICEPSVENDOR'] = bicepsvendor_value[3:]
    print('element_values:############',element_values)
    # Now 'BIC_VENDOR_FACILITY' and updated 'BICEPSVENDOR' are present in element_values
    # Replace column names with the ones from tag_to_column
    mapped_values = {tag_to_column[key]: value for key, value in element_values.items() if key in tag_to_column}
    # Include additional columns in the mapping
    mapped_values['REQUEST_ID'] = request_id
    mapped_values['CREATION_DATE'] = current_timestamp
    mapped_values['LAST_UPDATE_DATE'] = current_timestamp
    mapped_values['CREATED_BY'] = -1
    mapped_values['LAST_UPDATED_BY'] = -1
    print('mapped_values:########0',mapped_values)
    # Generate SQL INSERT statement
    columns_str = ",".join(mapped_values.keys())
    values_str = ",".join(f"'{v}'" for v in mapped_values.values())

    insert_query = f"INSERT INTO {table_name} ({columns_str}) VALUES ({values_str});"
    print(insert_query)

# Establish a database connection and execute the INSERT statement
    with psycopg2.connect(**db_params) as conn:
        with conn.cursor() as cursor:
            cursor.execute(insert_query)

        print(f"Record inserted: {mapped_values}")

# Establish a database connection and execute the INSERT statement
with psycopg2.connect(**db_params) as conn:
    with conn.cursor() as cursor:
    # Execute the SQL statement
        
        # Execute the SQL statement to select all rows from the table
        cursor.execute(f"SELECT * FROM XXPO_ORCL2CSPOMS_VEND_STG where PROCESS_STATUS ='N'")

        # Fetch all rows into a list of tuples
        result_rows = cursor.fetchall()

        # Get the column names from the cursor description
        column_names = [desc[0] for desc in cursor.description]

        # Create a DataFrame using the result and column names
        stg_df = pd.DataFrame(result_rows, columns=column_names)

        # Print or use the DataFrame as needed
        print(stg_df)
        # Extract the 'bic_vendor_number' column and convert it to a list
        bic_vendor_numbers_list = stg_df['bic_vendor_number'].tolist()

        # Convert the list of strings to a list of integers
        bic_vendor_numbers_list = list(map(int, bic_vendor_numbers_list))

        # Print or use the list as needed
        print(bic_vendor_numbers_list)

conn = psycopg2.connect(**db_params)
# Create a cursor object to execute SQL queries
cur = conn.cursor()
# Print or use the new value as needed
# print("New CS_VENDOR_ID:", new_cs_vendor_id)

# Convert the list of numbers to a string with quotes for the SQL query
vendor_numbers_str = ', '.join([f"'{num}'" for num in bic_vendor_numbers_list])


# Construct the SQL query
sql_query = f"SELECT * FROM APPS.XXAP_VENDOR_MASTER WHERE VENDOR_NBR IN ({vendor_numbers_str})"

oracle_df = pd.read_sql(sql_query, connection)
oracle_df['VENDOR_NBR'] = oracle_df['VENDOR_NBR'].str.zfill(6)
# Display the DataFrame
print(oracle_df.head())

# Merge oracle_df with stg_df based on the common column 'bic_vendor_number' and 'VENDOR_NBR'
combined_df = pd.merge(stg_df, oracle_df, how='left', left_on='bic_vendor_number', right_on='VENDOR_NBR')

#############################################################################
# Step 1: Retrieve the last_run from PostgreSQL
cur.execute("SELECT profile_value FROM cspoms_common_profiles WHERE profile_name = 'I1619_CSPOMS_RLX_BROKER_VENDOR'")
result = cur.fetchone()
print(f'TIme in the Profile Value',result)

# Retrieve the current timestamp from the database
profile_update_query = "SELECT CURRENT_TIMESTAMP"
cur.execute(profile_update_query)
current_timestamp = cur.fetchone()[0]
print(f'Time it is going to update next',current_timestamp)
# Convert the retrieved value to a datetime object
current_timestamp_dt = datetime.strptime(str(current_timestamp), "%Y-%m-%d %H:%M:%S.%f%z")

# Format the datetime object to match the desired format
current_timestamp_str = current_timestamp_dt.strftime("%Y-%m-%d %H:%M:%S")

if result:
    # Step 2: Convert the retrieved value to a datetime object
    last_run_str = result[0]  # Assuming the result is a string
    # Remove the UTC offset before parsing
    # Remove the UTC offset and microseconds before parsing
    last_run_str_without_offset = last_run_str.split('+')[0].split('.')[0]
    last_run_datetime = datetime.strptime(last_run_str_without_offset, "%Y-%m-%d %H:%M:%S")
    # Step 3: Format the datetime object to match the Oracle date format
    last_run_formatted = last_run_datetime.strftime("%Y-%m-%d %H:%M:%S")

    # Step 4: Use the formatted date in the Oracle SQL query
    # delta_run_query = f"SELECT * FROM XXAP_VENDOR_MASTER WHERE last_update_date > TO_DATE('{last_run_formatted}', 'YYYY-MM-DD HH24:MI:SS')"
    delta_run_query = f"SELECT * FROM XXAP_VENDOR_MASTER  WHERE ROWNUM <1000"
    delta_df = pd.read_sql(delta_run_query, connection)
    print('@@@@@@@delta_df', delta_df.columns)
    if not delta_df.empty:
        print('@@@@@@@@@@@@@combined_df,',combined_df.columns)
        delta_df['VENDOR_NBR'] = oracle_df['VENDOR_NBR'].str.zfill(6)
        # Merge delta_df with combined_df based on 'VENDOR_NBR'
        final_df = pd.merge(combined_df, delta_df, on='VENDOR_NBR', how='outer', suffixes=('', ''))
        print('final_df', final_df)
        # final_df = pd.merge(combined_df, delta_df, how='left', left_on='bic_vendor_number', right_on='VENDOR_NBR')
    else:
        print('No rows to fetch in delta logic from the last run')
        final_df = combined_df
        print('final_df', final_df)
    
else:
    print("No Profile value in cspoms_common_profiles for I1619_CSPOMS_RLX_BROKER_VENDOR. taking full records")
    oracle_query = f"SELECT * FROM XXAP_VENDOR_MASTER WHERE ROWNUM <10"



# Display the merged DataFrame
print(final_df)
# Define a custom function to determine LOAD_BUILDING
def determine_load_building(row):
    try:
        if (
            row['DIV_VNDR_FLG'] is not None and not pd.isna(row['DIV_VNDR_FLG']) or
            row['VENDOR_FOODLINK_FLG'] is not None and not pd.isna(row['VENDOR_FOODLINK_FLG']) or
            row['VENDOR_FREIGHT_FLG'] is not None and not pd.isna(row['VENDOR_FREIGHT_FLG'])
        ):
            return 'F'
        else:
            return 'B'
    except Exception as e:
        print(f"Error in determine_load_building function: {e}")
        print(f"Row details: {row}")
        raise  # Reraise the exception to see the traceback

# Apply the custom function to create the LOAD_BUILDING column
final_df['LOAD_BUILDING'] =  final_df.apply(determine_load_building, axis=1)
# Update 'VENDOR_TYPE' column based on conditions
final_df.loc[ final_df['VENDOR_TYPE'] == 'Broker', 'VENDOR_TYPE'] = 'B'
final_df.loc[final_df['VENDOR_TYPE'] == 'Buyer Vendor', 'VENDOR_TYPE'] = 'V'

# Duplicate the 'SHIP_TO_LOCATION_NAME' column as 'FACILITY_NAME'
final_df = final_df.assign(FACILITY_NAME=final_df['SHIP_TO_LOCATION_NAME'])
# CREATION_DATE	 :  	PREPAYADD_BUY_RATE_CHANGED
final_df = final_df.assign(PREPAYADD_BUY_RATE_CHANGED= final_df['CREATION_DATE'])
final_df = final_df.assign(BACKHAUL_BUY_RATE_CHANGED= final_df['CREATION_DATE'])
final_df = final_df.assign(FREIGHT_BILL_BUY_RATE_CHANGED= final_df['CREATION_DATE'])
# BIC_SSA_FLAG	 : 	SSA_RATE_BASIS_GROSS_FLAG
final_df = final_df.assign(SSA_RATE_BASIS_GROSS_FLAG= final_df['bic_ssa_flag'])
final_df = final_df.assign(MAXIMUM_BKT_TYPE= final_df['VENDOR_ORDER_UNIT'])
#     'VENDOR_ORDER2_UNIT':'MAX_BKT_TYPE_2ND'
final_df = final_df.assign(MAX_BKT_TYPE_2ND= final_df['VENDOR_ORDER2_UNIT'])
# 	,'bic_backhaul_type'	 : 	'BACKHAUL_BUY_RATE_TYPE'
final_df = final_df.assign(BACKHAUL_BUY_RATE_TYPE= final_df['bic_backhaul_type'])
#     ,'bic_flag_prepay_and_add'	 : 	'PRE_PAY_ADD_FLAG'
final_df = final_df.assign(PRE_PAY_ADD_FLAG= final_df['bic_flag_prepay_and_add'])

# # Rename columns
merged_df = final_df.rename(columns={
    'VENDOR_SITE_ID': 'CS_VEND_SITE_ID',
    'VENDOR_NBR': 'VENDOR_NUMBER',
    'BUYER_VENDOR_NAME': 'VENDOR_NAME',
    'bic_vendor_facility': 'VENDOR_FACILITY',
    'BICEPS_BUYER_CODE': 'BUYER_ID',
    'bic_warehouse_code': 'WAREHOUSE_CODE',
    'VENDOR_FOODLINK_FLG': 'FOOD_LINK_FLAG',
    'VENDOR_BROKER_NBR': 'BROKER_NUMBER',
    'VENDOR_BROKER_NAME': 'BROKER_NAME',
    'bic_flag_truck': 'TRUCK_FLAG',
    'bic_flag_rail': 'RAIL_FLAG',
    'bic_afe_contact': 'PO_TRANSMISSION_TYPE',  
    'bic_afe_fax_number': 'PO_TRANSMISSION_VALUE',
    'bic_freight_allow': 'FREIGHT_ALLOW_BUY_RATE',
    'LEAD_TIME_CALENDAR_IND' : 'LEAD_TIME_STATED_TYPE',
    'VENDOR_LEAD_DAYS' : 'LEAD_TIME_STATED_DAYS',
    'VENDOR_MIN_ORDER_SIZE' : 'MINIMUM_BKT_0',
    'VENDOR_ORDER_UNIT' : 'MINIMUM_BKT_0_TYPE',
    'VENDOR_MAX_ORDER_SIZE' : 'MAXIMUM_BKT',
    'VENDOR_MIN_ORDER2_SIZE':'MIN_BKT_2ND',
    'VENDOR_ORDER2_UNIT':'MIN_BKT_TYPE_2ND',
    'VENDOR_MAX_ORDER2_SIZE':'MAX_BKT_2ND'
    ,'VENDOR_DLVR_MONDAY_FLG' :  'FIXED_REVIEW_MON'
    ,'VENDOR_DLVR_TUESDAY_FLG' :  'FIXED_REVIEW_TUE'
    ,'VENDOR_DLVR_WEDNESDAY_FLG' :  'FIXED_REVIEW_WED'
    ,'VENDOR_DLVR_THURSDAY_FLG' :  'FIXED_REVIEW_THU'
    ,'VENDOR_DLVR_FRIDAY_FLG' :  'FIXED_REVIEW_FRI'
    ,'VENDOR_DLVR_SATURDAY_FLG' :  'FIXED_REVIEW_SAT'
    ,'VENDOR_DLVR_SUNDAY_FLG' :  'FIXED_REVIEW_SUN'
    ,'bic_order_sequence' : 'ORDER_SEQUENCE'
    ,'FILTER_VENDOR_FLG' : 'ORDER_FILTER_FLAG'
    ,'bic_freight_allow'	 :  	'FREIGHT_ALLOW_BUY_RATE'
    ,'bic_freight_allow_type'	 :  	'FREIGHT_ALLOW_BUY_RATE_TYPE'
    ,'creation_date'	 :  	'FREIGHT_ALLOW_BUY_RATE_CHANGED'
    ,'bic_backhaul_type'	 :  	'PREPAYADD_BUY_RATE_TYPE'
    ,'bic_freightbill'	 : 'FREIGHT_BILL_BUY_RATE'
	,'bic_freightbill_type'	 : 	'FREIGHT_BILL_BUY_RATE_TYPE'
	,'bic_backhaul'	 : 	'BACKHAUL_BUY_RATE'
    ,'bic_ssa_flag'	 : 	'SSA_RATE_BASIS_NET_FLAG'
    ,'bic_flag_prepaid'	 : 	'PRE_PAY_FLAG'
    ,'bic_flag_freight_bill'	 : 	'FREIGHT_BILL_FLAG'
    ,'bic_flag_backhaul'	 : 	'BACKHAUL_FLAG'
    ,'bic_external_backhaul'	 : 	'EXTERNAL_BACKHAUL_FLAG'
    ,'bic_ssa_allow_percent'	 : 	'SSA_RATE'
    ,'bic_ssa_allow_basis'	 : 	'SSA_RATE_TYPE'
    ,'bic_ssa_allow_account'	 : 	'SSA_RATE_BASIS_ACC'
    ,'VENDOR_DISC_PCT'	 : 	'CASH_DISC_TERMS'
    ,'VENDOR_DISCOUNT_DAYS_NBR'	 : 	'CASH_DISC_TERMS_DAYS'
    ,'VENDOR_NET_DAYS'	 : 	'CASH_DISC_TERMS_NET'

   
})
#     ,'bic_ssa_flag'	 : 	'SSA_RATE_BASIS_NET_FLAG'
#     ,'bic_flag_prepay_and_add'	 :  	'PREPAYADD_BUY_RATE'
# Display the resulting DataFrame
# print(merged_df)
                             
print(merged_df)
merged_df = merged_df.loc[:, ~merged_df.columns.duplicated()]





# Construct the DELTA LOGIC query
# sql_query = f"SELECT * FROM APPS.XXAP_VENDOR_MASTER WHERE LAST_UPDATE_DATE > ({vendor_numbers_str})"

# oracle_df = pd.read_sql(sql_query, connection)


# print(merged_df)




#     cur.execute(
#         """UPDATE  XXPO_ORCL2CSPOMS_VEND_STG
#            SET PROCESS_STATUS = 'P'
#            where PROCESS_STATUS ='N'
#         """
#     )
        # Use the formatted date to update the profile_value
        update_profile_query = f"UPDATE cspoms_common_profiles SET profile_value = '{current_timestamp_str}' WHERE profile_name = 'I1619_CSPOMS_RLX_BROKER_VENDOR'"
        cur.execute(update_profile_query)
        print(f'Used the formatted date to update the profile_value Done {current_timestamp_str}')
# Commit changes and close the connection
merged_df = merged_df.fillna('')
upsert_vendor_master_and_childs(merged_df,cur)

conn.commit()
cur.close()
conn.close()

IndentationError: unexpected indent (<ipython-input-1-31094eb6da21>, line 409)

In [13]:

combined_df['DIV_VNDR_FLG']

0    None
1    None
2    None
3    None
4    None
5    None
6    None
7    None
8    None
9    None
Name: DIV_VNDR_FLG, dtype: object

In [23]:
for i in final_df.columns:
    print(i)

stg_seq_id
bic_vendor_facility
bic_vendor_number
bic_buyer_number
bic_crp_level
bic_status
bic_po_recommended_ind
bic_afe_type1
bic_afe_fax_number
bic_afe_contact
bic_minimum_quantity
bic_minimum_quantity_sign
bic_minimum_type
bic_bracket_quantity
bic_bracket_quantity_sign
bic_current_bkt_number
bic_current_bkt_quantity
bic_current_bkt_quantity_sign
bic_current_bkt_type
bic_maximum_quantity
bic_maximum_quantity_sign
bic_maximum_type
bic_order_interval_weeks
bic_order_interval_weeks_sign
bic_lead_time_stated_weeks
bic_lead_time_stated_wks_sign
bic_fixed_review_day1
bic_fixed_review_day2
bic_fixed_review_day3
bic_fixed_review_day4
bic_fixed_review_day5
bic_fixed_review_day6
bic_fixed_review_day7
bic_target_service_level
bic_target_service_level_sign
bic_name
bic_address_1
bic_city
bic_state
bic_zip
bic_phone
bic_contact
bic_transfer_match_type
bic_payable_vendor_nbr
bic_lead_time_weeks_type
bic_duns_remit_no
bic_duns_remit_suffix
bic_facility_ship_to
bic_pre_sched_flag
bic_broker_number
